Hand Written Digit Recognition using MNIST Dataset


In [16]:
#importing required tools
import numpy as np
import h5py
from matplotlib import pyplot as plt

#Keras utilities
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import KFold

#import dataSet
from keras.datasets import mnist

Sequence of step for design of model and Evaluation
1. Loading of the dataset
2. Preparation of dataset
3. Defining Model
4. Evaluation of Model
5. Presentation of results

In [19]:
#Function for loading dataset
def load_dataset():
    (trainX,trainY),(testX,testY)=mnist.load_data()
    #Reshaping dataset
    trainX=trainX.reshape((trainX.shape[0],28,28,1))
    testX=testX.reshape((testX.shape[0],28,28,1))
    #one_hot encoding of labels
    trainY = to_categorical(trainY)
    testY = to_categorical(trainY)
    return trainX, trainY, testX, testY

In [20]:
#Function to prepare dataset
def prep_dataset(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    train_norm = train_norm/255.0
    test_norm = test_norm/255.0
    return train_norm, test_norm

In [27]:
#Defining Model
def def_model():
    model = Sequential(
        [
            Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(100, activation='relu', kernel_initializer='he_uniform'),
            Dense(10, activation='softmax')
        ]
    )
    opt = SGD(learning_rate=0.01, momentum=0.9) #Stochastic grad. desc.
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

To do the evaluation we can use the k fold cross validation taking k=5 such that each test set will be approx 20%
of the total input dataset.K=5 is chosen so as to rather not be too large or be too big to take up a lot of time

In [43]:
#Evaluation of Model
def eval_model(dataX, dataY, n_fold=5):
    scores, histories = list(), list()
    kfold = KFold(n_fold, shuffle=True, random_state=1)
    
    #enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        model = def_model()
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('>%.3f' %(acc*100.0))
        scores.append(acc)
        histories.append(history)
    return scores, histories

In [23]:
#function for analysis of results
def analyze(histories):
    for i in range(len(histories)):
        plt.subplot(2,1,1)
        plt.title('Cross Entropy Loss')
        plt.plot(histories[i].history['loss'],color='blue',label='train')
        plt.plot(histories[i].history['val_loss'],color='orange',label='test')
                
        plt.subplot(2,1,2)
        plt.title('Classification Accuracy')
        plt.plot(histories[i].history['accuracy'],color='blue',label='train')
        plt.plot(histories[i].history['val_accuracy'],color='orange',label='test')
    plt.show()

In [24]:
#summarizing the performance
def summary(scores):
    print("accuracy: mean=%.3f std=%.3f,n=%d" %(np.mean(scores)*100,np.std(scores)*100,len(scores)))
    plt.boxplot(scores)
    plt.show()


In [ ]:


#finalizing by calling all the functions
def run():
    trainx,trainy,testx,testy=load_dataset()
    trainx,testx=prep_dataset(trainx,testx)
    scores,histories=eval_model(trainx,trainy)
    analyze(histories)
    summary(scores)



In [45]:
run()

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (60000, 10, 2)
